In [22]:
import pandas as pd
import subprocess
import json
import sys

pd.set_option('max_colwidth',250)
pd.set_option('display.max_rows',90)
sys.setdefaultencoding('utf-8')

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

Explorando o Catalogo

In [2]:
cat = pd.read_csv("catalogo_de_dados.csv", on_bad_lines='skip',sep=";")

In [3]:
categoriasSenadores = cat.query("nom_sub_categoria == 'Senadores'")

In [4]:
senadoresExercicio = categoriasSenadores.query("des_grupo_dados == 'Senadores em exercício'")

In [5]:
conjSenadoresExercicio = senadoresExercicio.query("nom_conjunto_dados == 'Senadores em exercício' ")

Capturando Parlamentares em Exercicio

In [12]:
comando = 'curl https://legis.senado.leg.br/dadosabertos/arquivos/ListaParlamentarEmExercicio.json'
resultado = subprocess.run(comando, shell=True, text=True, capture_output=True)
dados = json.loads(resultado.stdout)
dados = pd.DataFrame(dados)
dados = (dados.loc["Parlamentares"]["ListaParlamentarEmExercicio"])
dados = pd.DataFrame(dados["Parlamentar"])
saida = pd.DataFrame()
for index,row in dados.iterrows():
    IdentificacaoParlamentar = pd.DataFrame(row["IdentificacaoParlamentar"])
    Mandato = pd.DataFrame(row["Mandato"])
    concatenar = pd.concat([IdentificacaoParlamentar,Mandato])
    saida = pd.concat([saida,concatenar])
# dados = pd.json_normalize(dados)
# dados = resultado.stdout
# dados = pd.read_json(dados, orient='index')

TypeError: DataFrame.__init__() got an unexpected keyword argument 'encoding'

In [9]:
IdentificacaoParlamentar = dados["IdentificacaoParlamentar"]
Mandato = dados["Mandato"]

In [10]:
dadosTMP = pd.DataFrame() 
for i in range(0,81):
    tmpIP = pd.DataFrame(IdentificacaoParlamentar[i])
#     mpM = pd.DataFrame(Mandato[i])
    tmpIP = tmpIP.reset_index()
#     mpM = tmpM.reset_index()
    tmpTel = tmpIP.iloc[0]["Telefones"]
    try:
        tmpTels = pd.DataFrame(tmpTel, index=["NumeroTelefone","OrdemPublicacao","IndicadorFax"])
    except:
        tmpTels = pd.DataFrame(tmpTel[0], index=["NumeroTelefone","OrdemPublicacao","IndicadorFax"])
        
    tmpIP.loc[0, "Telefones"] = tmpTels.loc["NumeroTelefone", "NumeroTelefone"]
    tmpIP.loc[0, "CodigoBloco"] = tmpIP.loc[1, "Bloco"]
    tmpIP.loc[0, "NomeBloco"] = tmpIP.loc[2, "Bloco"]
    tmpIP.loc[0, "NomeApelido"] = tmpIP.loc[3, "Bloco"]
    tmpIP.loc[0, "DataCriacao"] = tmpIP.loc[4, "Bloco"]
    tmpIP.drop([1,2,3,4], inplace=True)
    #mpIP.join(tmpM)
    dadosTMP = pd.concat([tmpIP,dadosTMP])
    dados = dadosTMP.drop(columns="index").reset_index().drop(columns="index")

In [11]:
dados.to_csv("../Dados/Senadores.csv",encoding='utf-8-sig')